In [19]:
import allel
import zarr
import re
import sys
import pandas

In [2]:
# location of the ag1000g data
chromosome = 'X'
vcf_path = '../../data/ag1000/chrX_36Ag_allsites_filtered.vcf.gz'
zarr_path = 'data/zarr/' + re.sub('.*/|.vcf.gz', '', vcf_path) + '.zarr'

In [3]:
# generate the zarr array 
allel.vcf_to_zarr(vcf_path, zarr_path, group=chromosome, fields='*', log=sys.stdout, overwrite=True)
callset = zarr.open_group(zarr_path, mode='r')

[vcf_to_zarr] 65536 rows in 0.90s; chunk in 0.90s (72710 rows/s); X :68426
[vcf_to_zarr] 131072 rows in 5.30s; chunk in 4.40s (14895 rows/s); X :136256
[vcf_to_zarr] 196608 rows in 7.53s; chunk in 2.23s (29388 rows/s); X :204165
[vcf_to_zarr] 262144 rows in 9.55s; chunk in 2.02s (32482 rows/s); X :272351
[vcf_to_zarr] 327680 rows in 11.55s; chunk in 2.00s (32731 rows/s); X :340854
[vcf_to_zarr] 393216 rows in 13.45s; chunk in 1.90s (34482 rows/s); X :412253
[vcf_to_zarr] 458752 rows in 15.65s; chunk in 2.20s (29854 rows/s); X :482969
[vcf_to_zarr] 524288 rows in 17.65s; chunk in 2.01s (32653 rows/s); X :555551
[vcf_to_zarr] 589824 rows in 19.60s; chunk in 1.94s (33720 rows/s); X :630479
[vcf_to_zarr] 655360 rows in 21.56s; chunk in 1.96s (33355 rows/s); X :704927
[vcf_to_zarr] 720896 rows in 23.55s; chunk in 1.98s (33032 rows/s); X :777494
[vcf_to_zarr] 786432 rows in 26.47s; chunk in 2.92s (22439 rows/s); X :849879
[vcf_to_zarr] 851968 rows in 29.32s; chunk in 2.85s (22990 rows/s); X 

In [24]:
# read in the list of samples/populations
pop_file_path = 'Ag1000_sampleIDs_popfile.txt'
poppanel = pandas.read_csv(pop_file_path, sep='\t', usecols=[0,1], names=['ID', 'Population'])
#poppanel = pandas.read_csv(args.populations, sep='\t', usecols=[0,1], names=['ID', 'Population'])
poppanel.head()

# get a list of samples from the callset
samples = callset[chromosome + '/samples'][:]
samples_list = list(samples)
#print('VCF samples:', samples_list)

samples_callset_index = [samples_list.index(s) for s in poppanel['ID']]

poppanel['callset_index'] = samples_callset_index

# use the popindices dictionary to keep track of the indices for each population
popindices={}
popnames = poppanel.Population.unique()
for name in popnames:
    popindices[name] = poppanel[poppanel.Population == name].callset_index.values

In [7]:
# the genotype calls
# recode the gt matrix as a Dask array (saves memory)
gt_dask = allel.GenotypeDaskArray(callset[chromosome + '/calldata/GT'])
gt_array = allel.GenotypeArray(gt_dask).to_packed()
gt_array = allel.GenotypeArray.from_packed(gt_array)
pos_array = allel.SortedIndex(callset[chromosome + '/variants/POS'])

In [51]:
# subset the genotypes for KES and BFS

gt_pop = gt_array.take(popindices['KES'], axis=1)
ac_kes = gt_pop.count_alleles()
pi, windows, n_bases, counts = allel.windowed_diversity(pos_array, ac_kes, size = 10000) 

pi_file = "data/allel_KES_pi.txt"
outfile = open(pi_file, 'w')
outfile.write("window_pos_1" + "\t" + "window_pos_2" + "\t" + "sk_allel_avg_pi" + "\t" + "sk_allel_no_sites" + "\t" + "sk_allel_n_var" + "\n")

for i in range(len(pi)):
    #print(str(windows[i][0])  + "\t" + str(windows[i][1]) + "\t" + str(pi[i]) + "\t" + str(n_bases[i]) + "\t" + str(counts[i]))
    outfile.write(str(windows[i][0])  + "\t" + str(windows[i][1]) + "\t" + str(pi[i]) + "\t" + str(n_bases[i]) + "\t" + str(counts[i]) + "\n")

outfile.close()


In [52]:
gt_pop = gt_array.take(popindices['BFS'], axis=1)
ac_bfs = gt_pop.count_alleles()
pi, windows, n_bases, counts = allel.windowed_diversity(pos_array, ac_bfs, size = 10000) 

pi_file = "data/allel_BFS_pi.txt"
outfile = open(pi_file, 'w')
outfile.write("window_pos_1" + "\t" + "window_pos_2" + "\t" + "sk_allel_avg_pi" + "\t" + "sk_allel_no_sites" + "\t" + "sk_allel_n_var" + "\n")

for i in range(len(pi)):
    #print(str(windows[i][0])  + "\t" + str(windows[i][1]) + "\t" + str(pi[i]) + "\t" + str(n_bases[i]) + "\t" + str(counts[i]))
    outfile.write(str(windows[i][0])  + "\t" + str(windows[i][1]) + "\t" + str(pi[i]) + "\t" + str(n_bases[i]) + "\t" + str(counts[i]) + "\n")

outfile.close()

In [55]:
dxy, windows, n_bases, counts = allel.windowed_divergence(pos_array, ac_bfs, ac_kes, size = 10000)

dxy_file = "data/allel_KES_BFS_dxy.txt"
outfile = open(dxy_file, 'w')
outfile.write("window_pos_1" + "\t" + "window_pos_2" + "\t" + "sk_allel_avg_dxy" + "\t" + "sk_allel_no_sites" + "\t" + "sk_allel_n_var" + "\n")

for i in range(len(dxy)):
    #print(str(windows[i][0])  + "\t" + str(windows[i][1]) + "\t" + str(pi[i]) + "\t" + str(n_bases[i]) + "\t" + str(counts[i]))
    outfile.write(str(windows[i][0])  + "\t" + str(windows[i][1]) + "\t" + str(dxy[i]) + "\t" + str(n_bases[i]) + "\t" + str(counts[i]) + "\n")

outfile.close()